# HyDesign Sizing P2X Example

## Sizing wind, solar, batteries and electrolyzers for a hybrid power plant

In [1]:
# Install hydesign if needed
import importlib
if not importlib.util.find_spec("hydesign"):
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git   

In [2]:
from hydesign.assembly.hpp_assembly_P2X import hpp_model_P2X as hpp_model
from hydesign.Parallel_EGO import EfficientGlobalOptimizationDriver
from hydesign.examples import examples_filepath
import pandas as pd
import os

In [3]:
n_procs = os.cpu_count()
if n_procs > 2:
    n_procs -= 1
    n_doe = int(3 * n_procs)
else:
    n_procs -= 0
    n_doe = int(8 * n_procs)
print(n_doe, n_procs)

21 7


In [4]:
examples_sites = pd.read_csv(f'{examples_filepath}examples_sites.csv', index_col=0, sep=';')
ex_site = examples_sites.loc[4]
longitude = ex_site['longitude']
latitude = ex_site['latitude']
altitude = ex_site['altitude']
input_ts_fn = examples_filepath+ex_site['input_ts_fn']
sim_pars_fn = examples_filepath+ex_site['sim_pars_fn']
H2_demand_fn = examples_filepath+ex_site['H2_demand_col']

inputs = {
    'name': ex_site['name'],
    'longitude': longitude,
    'latitude': latitude,
    'altitude': altitude,
    'input_ts_fn': input_ts_fn,
    'sim_pars_fn': sim_pars_fn,
    'H2_demand_fn': H2_demand_fn,

    'opt_var': "NPV_over_CAPEX",
    'num_batteries': 10,
    'n_procs': n_procs,
    'n_doe': n_doe,
    'n_clusters': n_procs,
    'n_seed': 0,
    'max_iter': 10,
    'final_design_fn': 'hydesign_design_0.csv',
    'npred': 3e4,
    'tol': 1e-6,
    'min_conv_iter': 3,
    'work_dir': './',
    'hpp_model': hpp_model,
    'variables': {
    'clearance [m]':
        {'var_type':'design',
          'limits':[10, 60],
          'types':'int'
          },
        # {'var_type':'fixed',
        #   'value': 35
        #   },
     'sp [W/m2]':
        {'var_type':'design',
         'limits':[200, 360],
         'types':'int'
         },
    'p_rated [MW]':
        {'var_type':'design',
          'limits':[1, 10],
          'types':'int'
          },
        # {'var_type':'fixed',
        #  'value': 6
         # },
    'Nwt':
        {'var_type':'design',
          'limits':[0, 400],
          'types':'int'
          },
        # {'var_type':'fixed',
        #   'value': 200
        #   },
    'wind_MW_per_km2 [MW/km2]':
        # {'var_type':'design',
        #   'limits':[5, 9],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
          'value': 7
          },
    'solar_MW [MW]':
        # {'var_type':'design',
        #   'limits':[0, 400],
        #   'types':'int'
        #   },
        {'var_type':'fixed',
          'value': 200
          },
    'surface_tilt [deg]':
        # {'var_type':'design',
        #   'limits':[0, 50],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
          'value': 25
          },
    'surface_azimuth [deg]':
        # {'var_type':'design',
        #   'limits':[150, 210],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
          'value': 180
          },
    'DC_AC_ratio':
        # {'var_type':'design',
        #   'limits':[1, 2.0],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
          'value':1.0,
          },
    'b_P [MW]':
        # {'var_type':'design',
        #   'limits':[0, 100],
        #   'types':'int'
        #   },
        {'var_type':'fixed',
          'value': 50
          },
    'b_E_h [h]':
        # {'var_type':'design',
        #   'limits':[1, 10],
        #   'types':'int'
        #   },
        {'var_type':'fixed',
          'value': 6
          },
    'cost_of_battery_P_fluct_in_peak_price_ratio':
        # {'var_type':'design',
        #   'limits':[0, 20],
        #   'types':'float'
        #   },
        {'var_type':'fixed',
          'value': 10},
    'ptg_MW [MW]':
        {'var_type':'design',
          'limits':[1, 50],
          'types':'int'
          },
    }}

EGOD = EfficientGlobalOptimizationDriver(**inputs)

In [5]:
EGOD.run()





Sizing a HPP plant at France_good_wind:


Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302.0



Initial 21 simulations took 2.73 minutes
  Current solution -NPV_over_CAPEX = -1.116E+00
  Current No. model evals: 21

Update sm and extract candidate points took 0.26 minutes
Check-optimal candidates: new 17 simulations took 4.3 minutes
  Current solution -NPV_over_CAPEX = -1.260E+00
  Current No. model evals: 38
  rel_yopt_change = -1.14E-01
Iteration 1 took 4.58 minutes

Update sm and extract candidate points took 0.21 minutes
Check-optimal candidates: new 17 simulations took 2.41 minutes
  Current solution -NPV_over_CAPEX = -1.355E+00
  Current No. model evals: 53
  rel_yopt_change = -7.01E-02
Iteration 2 took 2.64 minutes

Update sm and extract candidate points took 0.2 minutes
Check-optimal candidates: new 17 simulations took 1.98 minutes
  Current solution -NPV_over_CAPEX = -1.363E+00
  Current No. model evals:

In [6]:
EGOD.result

,clearance [m],sp [W/m2],p_rated [MW],Nwt,wind_MW_per_km2 [MW/km2],solar_MW [MW],surface_tilt [deg],surface_azimuth [deg],DC_AC_ratio,b_P [MW],...,Apvp [km2],Rotor diam [m],Hub height [m],Number of batteries used in lifetime,Break-even H2 price [Euro/kg],Break-even PPA price [Euro/MWh],Capacity factor wind [-],design obj,opt time [min],n_model_evals
France_good_wind,42.0,285.0,4.0,83.0,7.0,200.0,25.0,180.0,1.0,50.0,...,2.452,133.678827,108.839413,1.0,-1.594641,2.550869,0.403192,NPV_over_CAPEX,28.99,180
